In [ ]:
import torch
print(torch.__version__) 
# 1.6.0이 안나온다면 아래 코드 실행 필요 

1.9.0+cu102


In [ ]:
!pip3 install torch==1.6.0+cu101 torchtext==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
# 1회만 실행할 것 
# 실행 후 런타임 재실행 필요 

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0 MB 10 kB/s 
     |████████████████████████████████| 4.5 MB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0+cu101 which is incompatible.


In [ ]:
import torch, torchtext
print(torch.__version__)     # 1.6.0+cu101
print(torchtext.__version__) # 0.7.0

1.6.0+cu101
0.7.0


In [ ]:
# imdb data.zip 올리기
!mkdir imdb_data
!unzip imdb_data.zip -d imdb_data
!rm -rf imdb_data/test/.DS_Store
!rm -rf imdb_data/train/.DS_Store

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: imdb_data/train/pos/8890_10.txt  
  inflating: imdb_data/__MACOSX/train/pos/._8890_10.txt  
  inflating: imdb_data/train/pos/9866_7.txt  
  inflating: imdb_data/__MACOSX/train/pos/._9866_7.txt  
  inflating: imdb_data/train/pos/6823_10.txt  
  inflating: imdb_data/__MACOSX/train/pos/._6823_10.txt  
  inflating: imdb_data/train/pos/2109_9.txt  
  inflating: imdb_data/__MACOSX/train/pos/._2109_9.txt  
  inflating: imdb_data/train/pos/10546_9.txt  
  inflating: imdb_data/__MACOSX/train/pos/._10546_9.txt  
  inflating: imdb_data/train/pos/11434_7.txt  
  inflating: imdb_data/__MACOSX/train/pos/._11434_7.txt  
  inflating: imdb_data/train/pos/9112_9.txt  
  inflating: imdb_data/__MACOSX/train/pos/._9112_9.txt  
  inflating: imdb_data/train/pos/6923_10.txt  
  inflating: imdb_data/__MACOSX/train/pos/._6923_10.txt  
  inflating: imdb_data/train/pos/10991_9.txt  
  inflating: imdb_data/__MACOSX/train/pos/._10991_9.txt  
  inflating: imdb_data/tr

In [ ]:
import collections
import os

import nltk
nltk.download('punkt')
import torch
import tqdm.notebook as tqdm
from sklearn.model_selection import train_test_split
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from torchtext.experimental.vocab import Vocab
from torchtext.experimental.functional import sequential_transforms, totensor, vocab_func
from torchtext.experimental.datasets.text_classification import TextClassificationDataset

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# 하이퍼파라미터
num_epochs = 10
batch_size = 50
num_classes = 2
max_length = 256
vocab_size = 50000
embedding_dim = 300
filter_sizes = (3, 4, 5)
filter_counts = (100, 100, 100)
dropout_rate = 0.0
learning_rate = 0.001
evaluate_per_steps = 100

In [ ]:
def tokenize(text, max_length):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = tokens[:max_length]
    tokens.extend(['<pad>'] * (max_length - len(tokens)))
    return tokens

In [ ]:
def make_vocab(text, vocab_size):
    counts = collections.Counter()
    for tokens in text:
        for token in tokens:
            counts[token] += 1
    _, max_count = counts.most_common(1)[0]
    counts['<pad>'] = max_count + 2
    counts['<unk>'] = max_count + 1
    vocab = Vocab(collections.OrderedDict(counts.most_common(vocab_size)))
    return vocab

In [ ]:
def load_dataset(directory, dev_ratio=None, using_vocab=None):
    print(f'loading files in {directory}')
    text = []
    labels = []
    classes = os.listdir(directory) # [neg, pos]
    for directory_name in classes:
        for filename in tqdm.tqdm(os.listdir(f'{directory}/{directory_name}'), desc=f'loading {directory_name}'):
            with open(f'{directory}/{directory_name}/{filename}', encoding='utf-8') as f:
                tokens = tokenize(f.read(), max_length)
                text.append(tokens)
                labels.append(directory_name)

    if dev_ratio is not None:
        text, dev_text, labels, dev_labels = train_test_split(text, labels, test_size=0.1)

    if using_vocab is None:
        using_vocab = make_vocab(text, vocab_size)

    text_transform = sequential_transforms(
        vocab_func(using_vocab),
        totensor(torch.long)
    )
    label_map = {name: index for index, name in enumerate(classes)}
    print(label_map)
    label_transform = sequential_transforms(
        lambda label: label_map[label],
        totensor(torch.long)
    )

    dataset = TextClassificationDataset(list(zip(labels, text)), using_vocab, (label_transform, text_transform))

    if dev_ratio is not None:
        dev_dataset = TextClassificationDataset(list(zip(dev_labels, dev_text)), using_vocab, (label_transform, text_transform))
        return dataset, dev_dataset
    else:
        return dataset

In [ ]:
class TextCnn(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, filter_sizes, filter_counts, dropout_rate):
        super().__init__()

        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        layers = []
        # fs: [3, 4, 5]
        # fc: [100, 100, 100]
        #  <pad>  <pad>   i like  it  <pad>  <pad>
        for size, count in zip(filter_sizes, filter_counts):
            layers.append(nn.Conv1d(in_channels=embedding_dim, out_channels=count, kernel_size=size, padding=size - 1))

        self.conv = nn.ModuleList(layers)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(in_features=sum(filter_counts), out_features=num_classes)

    #  <pad>  <pad>   i   like  it it  <pad>  <pad>
    #    0       0    3    2    4   4    0      0   
    # emb: 5 x 300
    # emb[0], emb[0], emb[3] ....
    # -> out 8 x 300
    def forward(self, x):
        out = self.embedding(x)   # [batch, length] -> [batch, length, emb_dim]
        out.transpose_(1, 2) # [batch, length, emb_dim] -> [batch, emb_dim, length]
        features = []
        for layer in self.conv:
            feature = layer(out)   # -> [batch, filer_count, length + alpha]
            feature, _ = feature.max(dim=2)  # max-over-time pooling -> [batch, filer_count]
            features.append(feature)

        out = torch.cat(features, dim=1)  # -> [batch, sum(filter_count)]
        out = self.relu(out)
        out = self.fc(out)
        return out

In [ ]:
def evaluate(model, data_loader):
    with torch.no_grad():
        model.eval()
        num_corrects = 0
        num_total = 0
        for label, text in data_loader:
            label = label.to(device)
            text = text.to(device)

            output = model(text)
            predictions = output.argmax(dim=-1)
            num_corrects += (predictions == label).sum().item()
            num_total += text.size(0)

        return num_corrects / num_total

In [ ]:
# 데이터 준비 
train_dataset, dev_dataset = load_dataset('imdb_data/train', dev_ratio=0.1)
test_dataset = load_dataset('imdb_data/test', using_vocab=train_dataset.get_vocab())

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=8)
dev_loader   = DataLoader(dev_dataset, batch_size * 4, num_workers=8)  # * 4 : 빠른 inference를 위해 
test_loader  = DataLoader(test_dataset, batch_size * 4, num_workers=8) # * 4 : 빠른 inference를 위해 

# 모델 선언
model = TextCnn(num_classes, vocab_size, embedding_dim, filter_sizes, filter_counts, dropout_rate)
model.to(device)

# 옵티마이저, 로스 정의
optimizer = optim.Adam(model.parameters(), learning_rate)
criterion = nn.CrossEntropyLoss()

loading files in imdb_data/train


loading pos:   0%|          | 0/7500 [00:00<?, ?it/s]

loading neg:   0%|          | 0/7500 [00:00<?, ?it/s]

{'pos': 0, 'neg': 1}
loading files in imdb_data/test


loading pos:   0%|          | 0/500 [00:00<?, ?it/s]

loading neg:   0%|          | 0/500 [00:00<?, ?it/s]

{'pos': 0, 'neg': 1}


In [ ]:
# 학습 진행
steps = 0
max_dev_accuracy = 0.0

for epoch in tqdm.trange(num_epochs):
    progress = tqdm.tqdm(train_loader)
    for label, text in progress:
        model.train()
        steps += 1
        label = label.to(device)
        text = text.to(device)

        output = model(text)

        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress.set_description(f'train loss: {loss.item():.4f}')

        if steps % evaluate_per_steps == 0:
            print('***** evaluating on the dev set *****')
            dev_accuracy = evaluate(model, dev_loader)
            print(f'dev accuracy: {dev_accuracy:.4f}')
            if dev_accuracy > max_dev_accuracy:
                max_dev_accuracy = dev_accuracy
                print('achieve dev-best accuracy. saving.')
                torch.save(model.state_dict(), 'best_weight.pt')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.7833
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.7733


  0%|          | 0/270 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.8200
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.8380
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.8253


...

***** evaluating on the dev set *****
dev accuracy: 0.8540
***** evaluating on the dev set *****
dev accuracy: 0.8567
achieve dev-best accuracy. saving.
***** evaluating on the dev set *****
dev accuracy: 0.8513


  0%|          | 0/270 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.8540
***** evaluating on the dev set *****
dev accuracy: 0.8547
***** evaluating on the dev set *****
dev accuracy: 0.8567


  0%|          | 0/270 [00:00<?, ?it/s]

***** evaluating on the dev set *****
dev accuracy: 0.8520
***** evaluating on the dev set *****
dev accuracy: 0.8513
***** evaluating on the dev set *****
dev accuracy: 0.8513


In [ ]:
# 평가
print('***** evaluating dev-best on the test set *****')
model.load_state_dict(torch.load('best_weight.pt'))
test_accuracy = evaluate(model, test_loader)
print(f'test accuracy: {test_accuracy}')

***** evaluating dev-best on the test set *****
test accuracy: 0.875


In [ ]:
## 실제 IMDB 데이터 이용해 평가해보기 
def load_imdb(review, score, vocab): 
    print(f'loading imdb text and score data')
    with open(review) as f:
        text = [tokenize(line, max_length) for line in f.readlines()]
    with open(score) as f:
        score = [] 
        for real_scroe in f.readlines() : 
            if int(real_scroe) >= 6 : 
                score.append(0) # positive
            else : 
                score.append(1) # negative

    text_transform = sequential_transforms(
        vocab_func(vocab),
        totensor(torch.long)
    )
    label_transform = sequential_transforms(
        totensor(torch.long)
    )
    dataset = TextClassificationDataset(list(zip(score, text)), vocab, (label_transform, text_transform))
    return dataset
    
imdb_dataset = load_imdb('imdb_data/review.txt', 'imdb_data/score.txt', vocab=train_dataset.get_vocab())
imdb_loader = DataLoader(imdb_dataset, batch_size=100, shuffle=True, num_workers=8)

acc = evaluate(model, imdb_loader)
print(f'accuracy of imdb dataset : {acc:.4f}')

loading imdb text and score data
accuracy of imdb dataset : 0.8133
